In [1]:
# imports

import os
import re
import json
from dotenv import load_dotenv
from huggingface_hub import login
from openai import OpenAI
from pricer.items  import Item
from pricer.evaluator import evaluate

In [2]:

load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
dataset = "ed-donner/items_lite"
train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


In [4]:
openai = OpenAI()

# Data size

OpenAI recommends fine-tuning with a small population of 50-100 examples

so, i m goona go with 100 examples


In [5]:

fine_tune_train = train[:100]
fine_tune_validation = val[:50]

In [6]:
len(fine_tune_train)

100

# Step 1

Prepare our data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI

In [7]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [
        {"role": "user", "content": message},
        {"role": "assistant", "content": f"${item.price:.2f}"}
    ]

In [8]:
messages_for(fine_tune_train[0])

[{'role': 'user',
  'content': 'Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  \nCategory: Home Hardware  \nBrand: Schlage  \nDescription: A single‑piece oil‑rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  \nDetails: Designed for a 4" minimum center‑to‑center door prep, it offers a lifetime mechanical and finish warranty and comes ready for quick installation.'},
 {'role': 'assistant', 'content': '$64.30'}]

In [9]:
# Convert the items into a list of json objects - a "jsonl" string
# Each row represents a message in the form:
# {"messages" : [{"role": "system", "content": "You estimate prices...


def make_jsonl(items):
    result = ""
    for item in items:
        messages = messages_for(item)
        messages_str = json.dumps(messages)
        result += '{"messages": ' + messages_str +'}\n'
    return result.strip()

In [11]:
make_jsonl(fine_tune_train[:2])

'{"messages": [{"role": "user", "content": "Estimate the price of this product. Respond with the price, no explanation\\n\\nTitle: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  \\nCategory: Home Hardware  \\nBrand: Schlage  \\nDescription: A single\\u2011piece oil\\u2011rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  \\nDetails: Designed for a 4\\" minimum center\\u2011to\\u2011center door prep, it offers a lifetime mechanical and finish warranty and comes ready for quick installation."}, {"role": "assistant", "content": "$64.30"}]}\n{"messages": [{"role": "user", "content": "Estimate the price of this product. Respond with the price, no explanation\\n\\nTitle: Mini Electric Air Duster Fan  \\nCategory: Electronics  \\nBrand: Kica  \\nDescription: Ultra\\u2011compact 86,000\\u202fRPM electric air duster with 11\\u202fm/s wind speed for precise cleaning and inflation.  \\nDetails: Powered by a 9.99\\u202fWh motor, adjustable in four speed

In [12]:
# Convert the items into jsonl and write them to a file

def write_jsonl(items, filename):
    with open(filename, "w") as f:
        jsonl = make_jsonl(items)
        f.write(jsonl)

In [13]:
write_jsonl(fine_tune_train, "jsonl/fine_tune_train.jsonl")


In [20]:
write_jsonl(fine_tune_validation, "jsonl/fine_tune_validation.jsonl")


In [21]:
with open("jsonl/fine_tune_train.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

In [22]:
train_file

FileObject(id='file-2YUvYiSPBe6gc56uZzWrdH', bytes=55120, created_at=1771412640, filename='fine_tune_train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [23]:
with open("jsonl/fine_tune_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [24]:
validation_file

FileObject(id='file-UGAwTmsDLaViv7a9osRYmB', bytes=27637, created_at=1771412644, filename='fine_tune_validation.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

# Step 2

## And now time to Fine-tune!

In [25]:
openai.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=validation_file.id,
    model="gpt-4.1-nano-2025-04-14",
    seed=42,
    hyperparameters={"n_epochs": 1, "batch_size": 1},
    suffix="pricer"
)

FineTuningJob(id='ftjob-2xQpgeZot5lHiMZpvLatKArv', created_at=1771412814, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-kmhgHPXGNsqZIbxgdJIX8TyA', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-2YUvYiSPBe6gc56uZzWrdH', validation_file='file-UGAwTmsDLaViv7a9osRYmB', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1))), user_provided_suffix='pricer', usage_metrics=None, shared_with_openai=False, eval_id=None, internal_worker_backend=None)

In [26]:
openai.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-2xQpgeZot5lHiMZpvLatKArv', created_at=1771412814, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-kmhgHPXGNsqZIbxgdJIX8TyA', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-2YUvYiSPBe6gc56uZzWrdH', validation_file='file-UGAwTmsDLaViv7a9osRYmB', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1))), user_provided_suffix='pricer', usage_metrics=None, shared_with_openai=False, eval_id=None, internal_worker_backend=None)], has_more=False, object='list')

In [27]:
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id

In [28]:
job_id

'ftjob-2xQpgeZot5lHiMZpvLatKArv'

In [ ]:
openai.fine_tuning.jobs.retrieve(job_id)

In [29]:
openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

[FineTuningJobEvent(id='ftevent-iAmPu4gCfUHSU2MBzx78V4C7', created_at=1771412893, level='info', message='Fine-tuning job started', object='fine_tuning.job.event', data=None, type='message'),
 FineTuningJobEvent(id='ftevent-tHwAejtSXiTpMb3ITBHC74Ux', created_at=1771412892, level='info', message='Files validated, moving job to queued state', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-M6j7RRg4tnsLFhhprGzsWWcn', created_at=1771412815, level='info', message='Validating training file: file-2YUvYiSPBe6gc56uZzWrdH and validation file: file-UGAwTmsDLaViv7a9osRYmB', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-bb5yYlYBoC2HFLaYOoJ0p7E5', created_at=1771412814, level='info', message='Created fine-tuning job: ftjob-2xQpgeZot5lHiMZpvLatKArv', object='fine_tuning.job.event', data={}, type='message')]

### Testing our fine tuned model


In [49]:
fine_tuned_model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model

In [50]:
fine_tuned_model_name

'ft:gpt-4.1-nano-2025-04-14:personal:pricer:DAZzMzP2'

In [51]:

def test_messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [
        {"role": "user", "content": message},
    ]

In [52]:
test_messages_for(test[0])

[{'role': 'user',
  'content': 'Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Excess V2 Distortion/Modulation Pedal  \nCategory: Music Pedals  \nBrand: Old Blood Noise  \nDescription: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  \nDetails: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.'}]

In [53]:

def gpt_4__1_nano_fine_tuned(item):
    response = openai.chat.completions.create(
        model=fine_tuned_model_name,
        messages=test_messages_for(item),
        max_tokens=7
    )
    return response.choices[0].message.content

In [54]:
print(test[0].price)
print(gpt_4__1_nano_fine_tuned(test[0]))

219.0
$239.00


In [57]:
evaluate(gpt_4__1_nano_fine_tuned, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$76 $83 $0 $30 $69 $100 $23 $91 $1 $50 $529 $155 $8 $58 $14 $5 $15 $6 $32 $54 $55 $6 $345 $85 $212 $294 $45 $5 $431 $62 $40 $10 $281 $40 $149 $366 $147 $58 $98 $1 $35 $35 $18 $69 $40 $35 $72 $3 $101 $27 $7 $113 $31 $20 $712 $31 $18 $67 $36 $6 $132 $40 $4 $52 $239 $10 $65 $290 $75 $453 $8 $32 $140 $91 $45 $8 $16 $6 $13 $0 $29 $8 $262 $49 $10 $15 $63 $128 $84 $16 $1 $22 $8 $13 $12 $103 $9 $950 $25 $27 $23 $91 $53 $100 $17 $161 $13 $362 $5 $10 $41 $343 $76 $17 $54 $155 $50 $37 $55 $13 $29 $321 $16 $142 $1 $59 $15 $171 $40 $14 $282 $179 $1 $0 $85 $2 $32 $20 $53 $67 $26 $31 $216 $2 $126 $165 $5 $144 $180 $5 $10 $229 $34 $104 $23 $306 $129 $40 $119 $8 $210 $16 $118 $1 $302 $6 $309 $24 $13 $12 $9 $4 $235 $7 $64 $66 $3 $108 $62 $33 $176 $48 $136 $123 $16 $26 $96 $17 $33 $10 $5 $297 $1 $41 $47 $15 $44 $35 $19 $9 

In [59]:
import gradio as gr

In [71]:

SYSTEM_PROMPT = """
You are a product price estimation AI.
Respond ONLY with the estimated price number.
No explanation.
"""

In [72]:

def estimate_price(product_description, history):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": f"Estimate the price of this product:\n{product_description}"}
    ]

    response = openai.chat.completions.create(
        model=fine_tuned_model_name,
        messages=messages,
        max_tokens=10,
        temperature=0
    )

    return response.choices[0].message.content



In [ ]:


# 🔴 PUT YOUR REAL MODEL ID
MODEL = fine_tuned_model_name

def build_prompt(title, category, brand, description, details):

    prompt = f"""Estimate the price of this product. Respond with the price, no explanation

Title: {title}
Category: {category}
Brand: {brand}
Description: {description}
Details: {details}
"""
    return prompt



def estimate_price(title, category, brand, description, details):

    prompt = build_prompt(title, category, brand, description, details)

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=10,
        temperature=0
    )

    return response.choices[0].message.content.strip()


with gr.Blocks(theme=gr.themes.Soft(), title="AI Price Estimator") as demo:

    gr.Markdown(
        """
        # 💰 AI Product Price Estimator  
        Fine-tuned GPT model trained on product pricing dataset.

        Fill product details → click **Estimate Price**
        """
    )

    with gr.Row():
        title = gr.Textbox(label="Title")
        brand = gr.Textbox(label="Brand")

    with gr.Row():
        category = gr.Textbox(label="Category")
        details = gr.Textbox(label="Details")

    description = gr.Textbox(
        label="Description",
        lines=3
    )

    estimate_btn = gr.Button("Estimate Price 💰")

    output = gr.Textbox(
        label="Estimated Price",
        lines=1
    )

    estimate_btn.click(
        estimate_price,
        inputs=[title, category, brand, description, details],
        outputs=output
    )

    gr.Examples(
        examples=[
            [
                "Mini Electric Air Duster Fan",
                "Electronics",
                "Kica",
                "Ultra-compact electric air duster for cleaning",
                "86,000 RPM, lithium battery"
            ],
            [
                "Wooden Dining Table",
                "Furniture",
                "IKEA",
                "6 seater wooden dining table",
                "Solid oak wood"
            ]
        ],
        inputs=[title, category, brand, description, details]
    )

demo.launch()


/var/folders/5c/yyc3lwsx36zbbb6h7l_5yz3c0000gn/T/ipykernel_94988/2308586115.py:32: UserWarning:

The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.



* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
